In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix, log_loss
import numpy as np
from keras.layers import Dense, Dropout

%matplotlib inline 

Using TensorFlow backend.


In [2]:
data = pd.read_csv('clean_data.csv').reset_index()
data = data[['slope_of_peak_exercise_st_segment','resting_blood_pressure',
      'chest_pain_type','num_major_vessels','fasting_blood_sugar_gt_120_mg_per_dl',
     'resting_ekg_results','serum_cholesterol_mg_per_dl','oldpeak_eq_st_depression','sex' 
     ,'age','max_heart_rate_achieved', 'exercise_induced_angina', 'heart_disease_present',
     'quant_thal']]

data.head()

,slope_of_peak_exercise_st_segment,resting_blood_pressure,chest_pain_type,num_major_vessels,fasting_blood_sugar_gt_120_mg_per_dl,resting_ekg_results,serum_cholesterol_mg_per_dl,oldpeak_eq_st_depression,sex,age,max_heart_rate_achieved,exercise_induced_angina,heart_disease_present,quant_thal
0,1,128,2,0,0,2,308,0.0,1,45,170,0,0,0
1,2,110,3,0,0,0,214,1.6,0,54,158,0,0,0
2,1,125,4,3,0,2,304,0.0,1,77,162,1,1,0
3,1,152,4,0,0,0,223,0.0,1,40,181,0,1,1
4,3,178,1,0,0,2,270,4.2,1,59,145,0,0,1


In [3]:
#sns.pairplot(data, hue = 'heart_disease_present', diag_kind = 'hist')

In [4]:
vals = data.drop(columns = 'heart_disease_present')
labs = data['heart_disease_present']

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X = vals)
scaled_vals = scaler.transform(vals)



In [5]:
from sklearn.model_selection import train_test_split
X = scaled_vals
y = labs

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=101)

In [6]:
import keras
inputs = tf.keras.Input(shape = (13,))
x = layers.Dense(8, activation = 'linear')(inputs)
#x = layers.BatchNormalization(axis = -1)(x)
x = layers.Dropout(rate = 0.1)(x)
#x = layers.BatchNormalization(axis = 1)(x)
#x = layers.Dense(8, activation = 'relu')(x)

predictions = layers.Dense(2, activation='softmax')(x)
model = tf.keras.Model(inputs=inputs, outputs=predictions)


model.compile( optimizer = 'Adam',
             loss = keras.losses.SparseCategoricalCrossentropy(),
             metrics = [tf.keras.metrics.SparseCategoricalAccuracy()])



In [7]:
X_train, y_train = np.asarray(X_train), np.asarray(y_train)



model.fit(X_train, y_train, batch_size = 50, epochs = 50, verbose = 0)

In [8]:
def convert(preds):
    new_preds = []
    for pred in preds:
        if pred[0] > pred[1]:
            new_preds.append(0)
        else:
            new_preds.append(1)
    return new_preds

In [9]:
import numpy as np
X_test = np.asarray(X_test)
preds = model.predict(X_test)
preds = convert(preds)

print(classification_report(y_test,preds))
print('log loss: ',log_loss(y_test, preds))

              precision    recall  f1-score   support

           0       0.72      0.82      0.77        44
           1       0.80      0.70      0.74        46

    accuracy                           0.76        90
   macro avg       0.76      0.76      0.76        90
weighted avg       0.76      0.76      0.75        90

log loss:  8.442883082971962


# Grid Searching Hyper Parameters 

In [10]:
from keras.models import Sequential
from keras import optimizers as opt
from keras.wrappers.scikit_learn import KerasClassifier


In [15]:
from sklearn.model_selection import GridSearchCV
from keras import optimizers as opt

batch_size = [50,60,80,100]
epochs = [10,40,80,100]
num_neurons = [8,16,32,64,128]
activation = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
optimizer = [opt.Adam, opt.SGD, opt.RMSprop, opt.Adagrad, opt.Adadelta, opt.Adamax, opt.Nadam]
#optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
weight_constraint = [1, 2, 3, 4, 5]
dropout_rate = [0.0, 0.1, 0.2]





Doing a test for a 'Fast grid' to make sure code is working properly

In [12]:
def create_seq_model_fast():

    model = Sequential()

    model.add(Dense(16, input_dim = 13, activation = 'relu'))
    model.add(Dense(2, activation = 'softmax'))
    model.compile(optimizer= opt.Adam(), loss = keras.losses.SparseCategoricalCrossentropy(),
                  metrics = ['accuracy'])

    return model


    
    
model = KerasClassifier(build_fn=create_seq_model_fast, verbose=0)

grid = GridSearchCV(estimator=model, param_grid=param_grid_fast, n_jobs=-1, cv = 3)

grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

df = pd.DataFrame(grid_result.cv_results_)



NameError: name 'param_grid_fast' is not defined

Now for the actual model

In [16]:
def create_seq_model(activation, optimizer, init_mode, dropout_rate):

    model = Sequential()

    model.add(Dense(16, input_dim = 13, activation = activation, kernel_initializer = init_mode))
    model.add(Dropout(dropout_rate))
    model.add(Dense(2, activation = 'softmax', kernel_initializer = init_mode))
    model.compile(optimizer= optimizer, loss = keras.losses.SparseCategoricalCrossentropy(),
                  metrics = ['accuracy'])

    return model

## model = KerasClassifier(build_fn=create_seq_model, epochs = 100, batch_size = 50, verbose=0)
'''
param_grid = {'batch_size' : batch_size, 'epochs' : epochs, 
              'num_neurons':num_neurons, 'activation': activation, 
              'optimizer': optimizer, 'init_mode': init_mode, 'dropout_rate': dropout_rate,
             'weight_constraint': weight_constraint}
'''

param_grid = {'activation': activation, 
              'optimizer': optimizer, 'init_mode': init_mode, 'dropout_rate': dropout_rate,}

grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv = 3, verbose = 2)

grid_result = grid.fit(X_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

df = pd.DataFrame(grid_result.cv_results_)
df.to_csv('grid_results_accuracy.csv')

